# Image Recognition System

### An image recognition system based on CNN. The system classifies 10 types of images from cifar10 dataset

In [2]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
import keras
from keras.models import Sequential,model_from_json
from keras.preprocessing import image
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPool2D
from pathlib import Path

cifar10_class_names = {
    0:"Plane",
    1:"car",
    2:"Bird",
    3:"Cat",
    4:"Deer",
    5:"Dog",
    6:"Frog",
    7:"Horse",
    8:"Boat",
    9:"Truck"
}

## Load the dataset

In [3]:
(x_train, y_train), (x_test,y_test) = cifar10.load_data()

In [4]:
# Normalize the data to float values between 0-1.
# This improves the training process
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train = x_train/255
x_test = x_test/255

# Transform the labels to from single values to an array with 10 elements to match our CNN output
y_train = keras.utils.to_categorical(y_train,10)
y_test = keras.utils.to_categorical(y_test,10)


## Building the model

In [5]:
model = Sequential()

model.add(Conv2D(32,(3,3),padding='same',activation='relu',input_shape=(32,32,3)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512,activation='relu',input_shape=(32,32,3)))
model.add(Dropout(0.5))
model.add(Dense(10,activation="softmax"))

model.compile(
    loss="categorical_crossentropy",
    optimizer="Adam",
    metrics=["accuracy"]
)

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 15, 15, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 15, 15, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 13, 64)        36928     
                                                        

## Training the model
* NOTE - The last training process took about 54 minutes to complete. For that reason, I save the model for later use so I won't need to train it each time I work on this file 

In [6]:
model.fit(x_train,y_train,batch_size=32,epochs=30,validation_data=(x_test,y_test),shuffle=True)

Epoch 1/30
 337/1563 [=====>........................] - ETA: 1:17 - loss: 1.9229 - accuracy: 0.2891

KeyboardInterrupt: 

## Save the model

* The model is save in 2 file:
    1) json file that describes the model's structure (i.e. layers and connections)
    2) h5 file that saves the weights of the model after the training

In [ ]:

# Save the CNN structure in json
model_structure = model.to_json()
# Transform the file from json to text
f = Path("model_structure.json")
f.write_text(model_structure)
# Save the CNN weights
model.save("model_weights.h5",save_format="h5")

NameError: name 'model' is not defined

## Load the model

In [ ]:
# Read models structure
f = Path("model_structure.json")
saved_model_structure = f.read_text()
# Load model structure
saved_model = model_from_json(saved_model_structure)
# Load model weights
saved_model.load_weights("model_weights.h5")


## Use the saved model for predictions

In [ ]:
# Get an image
img = keras.utils.load_img("cat.png",target_size=(32,32))
# Transofrm it to an array
image_to_test = image.image_utils.img_to_array(img)/255
# expand to list of images (that's how the model was trained)
list_of_images = np.expand_dims(image_to_test,axis=0)
# Get predictions
results = model.predict(list_of_images)
# Get only the first (we only have 1 image)
image_result = results[0]
# Get the result with the most likelihood
most_likely = int(np.argmax(image_result))
class_label = cifar10_class_names[most_likely]

class_label

1/1 [==============================] - 0s 21ms/step


'Cat'